In [2]:
! pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=4c73939816c6595da5c0d52714590b4057ea4e5d04f561985f6f729ccb712808
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession


In [4]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("ZipExample") \
    .getOrCreate()

In [5]:
# Load the datasets into DataFrames
df1 = spark.read.option("header", "true").csv("/content/spam_ham_dataset.csv")
df2 = spark.read.option("header", "true").csv("/content/spam.csv")


In [6]:
# Add a row index to each DataFrame
from pyspark.sql.functions import monotonically_increasing_id
df1 = df1.withColumn("row_index", monotonically_increasing_id())
df2 = df2.withColumn("row_index", monotonically_increasing_id())


In [7]:
# Zip the two DataFrames together
zipped_df = df1.join(df2, "row_index").drop("row_index")


In [8]:
# Show the zipped DataFrame
zipped_df.show()

+--------------------+--------------------+--------------------+--------------------+----+--------------------+----+----+----+
|                 _c0|               label|                text|           label_num|  v1|                  v2| _c2| _c3| _c4|
+--------------------+--------------------+--------------------+--------------------+----+--------------------+----+----+----+
|                 605|                 ham|Subject: enron me...|                NULL| ham|Go until jurong p...|NULL|NULL|NULL|
|this is a follow ...| 4 / 3 / 00 { pre...|                NULL|                NULL| ham|Ok lar... Joking ...|NULL|NULL|NULL|
|flow data provide...|                NULL|                NULL|                NULL|spam|Free entry in 2 a...|NULL|NULL|NULL|
|please override p...|                NULL|                NULL|                NULL| ham|U dun say so earl...|NULL|NULL|NULL|
|activity you can ...|                NULL|                NULL|                NULL| ham|Nah I don't think...|

In [9]:
# Stop SparkSession
spark.stop()